In [1]:
'''
File name: project.ipynb
Author: ...Jose, Mohamed Ndoye, Raphael Strebel
Date created: 03/11/2019
Date last modified: ...
Python Version: 3.7.4
''';

<a id="up"></a>
# Food Inspections in Chicago

 - [Load Databases](#load-databases)
 - [Complete Datasets](#complete-datasets)
 - [Basic Statistics](#basic-stats)

In [16]:
# useful : https://www.sustainabilist.com/blog/chicago-data-analysis-a-internship-project

import pandas as pd
import geopandas as gpd

import vincent
vincent.core.initialize_notebook() 

from utils import constants as cst
from utils import clean_database
from utils import web_scraping_google_maps as ws
from utils import areas_handler

# Set auto-reload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id = 'load-databases'></a>
## Load Databases

In this section we load and clean the databases.

[Table of Contents](#up)

In [17]:
# Load the food inspections dataframe
food_inspections_DF = pd.read_csv(cst.FOOD_INSPECTIONS_PATH, sep = ',', header = 0, 
                   names = cst.FOOD_INSPECTIONS_COL_NAMES, index_col = None, error_bad_lines=False
                   )

In [19]:
# Clean the dataframe
food_inspections_DF = clean_database.drop_columns_with_one_value(food_inspections_DF)

# must complete missing lat/lng values

food_inspections_DF.head()

,inspection_id,DBA_name,AKA_name,license_num,facility_type,risk,address,city,zip,inspection_date,inspection_type,result,violations,lat,lng,location
0,2213280,UNION FULLBOARD,UNION FULLBOARD,2583580.0,Restaurant,Risk 1 (High),3469-3475 N CLARK ST,CHICAGO,60657.0,2018-08-21T00:00:00,License,Pass w/ Conditions,NaN,41.945290,-87.654804,"{'longitude': '-87.65480374883363', 'needs_rec..."
1,2213271,CHARTWELLS,CHARTWELLS,2609021.0,Restaurant,Risk 2 (Medium),2330 N HALSTED ST,CHICAGO,60614.0,2018-08-21T00:00:00,License,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.924568,-87.648875,"{'longitude': '-87.64887518194728', 'needs_rec..."
2,2213278,UNION FULLBOARD,UNION FULLBOARD,2583579.0,Restaurant,Risk 1 (High),3469-3475 N CLARK ST,CHICAGO,60657.0,2018-08-21T00:00:00,License,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.945290,-87.654804,"{'longitude': '-87.65480374883363', 'needs_rec..."
3,2213241,WALGREENS #15350,WALGREENS,2608770.0,Grocery Store,Risk 3 (Low),834 W ARMITAGE AVE,CHICAGO,60614.0,2018-08-21T00:00:00,License,Pass w/ Conditions,NaN,41.918268,-87.649873,"{'longitude': '-87.64987324323187', 'needs_rec..."
4,2213236,LONGACRE,LONGACRE,2609723.0,Restaurant,Risk 1 (High),1303-1309 W WILSON AVE,CHICAGO,60640.0,2018-08-21T00:00:00,License,Pass w/ Conditions,NaN,41.965218,-87.662586,"{'longitude': '-87.66258555303558', 'needs_rec..."


In [21]:
# Load the socio-economic indicators dataframe
socio_economic_DF = pd.read_csv(cst.SOCIO_ECONOMIC_INDICATORS_PATH, sep = ',', header = 0, 
                   names = cst.SOCIO_ECONOMIC_COL_NAMES, index_col = None, error_bad_lines=False
                   )

In [22]:
# Clean the dataframe
socio_economic_DF = clean_database.clean_socio_economic_df(socio_economic_DF)

socio_economic_DF.head()

,community_area_num,community_area_name,housing_crowded_perc,housholds_below_poverty_perc,aged_16_or_more_unemployed_perc,aged_25_or_more_without_high_school_diploma_perc,aged_under_18_or_over_64_perc,per_capita_income,hardship_idx
0,1,rogers park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2,west ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3,uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4,lincoln square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5,north center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [24]:
# Load the life expectancy dataframe
life_expectancy_DF = pd.read_csv(cst.LIFE_EXPECTANCY_PATH, sep = ',', header = 0, 
                   names = cst.LIFE_EXPECTANCY_COL_NAMES, index_col = None, error_bad_lines=False
                   )

In [25]:
# Clean the dataframe
life_expectancy_DF = clean_database.clean_socio_economic_df(life_expectancy_DF)

life_expectancy_DF.head()

,community_area_num,community_area_name,life_exp_1990,lower_95_perc_CI_1990,upper_95_perc_CI_1990,life_exp_2000,lower_95_perc_CI_2000,upper_95_perc_CI_2000,life_exp_2010,lower_95_perc_CI_2010,upper_95_perc_CI_2010
0,1,rogers park,70.9,69.9,71.9,73.1,72.2,74.1,77.3,76.3,78.2
1,2,west ridge,76.9,76.1,77.8,78.1,77.3,78.8,80.3,79.5,81.1
2,3,uptown,64.0,63.1,64.9,71.7,70.8,72.7,76.0,75.1,76.9
3,4,lincoln square,74.2,73.1,75.4,76.8,75.8,77.8,80.5,79.3,81.6
4,5,north center,73.4,72.1,74.7,77.9,76.6,79.1,81.5,80.1,82.8


<a id = 'complete-datasets'></a>
## Complete Datasets

### 2 problems : 
1. we only have the area name for the life exp. and the socio-eco DFs -> find the regions in sequence of lat/lng pairs that corresponds to the bounderies of an area. Then we can determine the region of the facility of the food_inspections dataframe and work only with the regions for the rest of the project (thoughts ?).
2. some entries in food_inspections_DF have no lat/lng pair -> must find it given their address

[Table of Contents](#up)

In [26]:
# merge socio-economic and life expectancy df's on the area number and names
socio_life_merged_DF = socio_economic_DF.merge(life_expectancy_DF, how="left", on=["community_area_num", "community_area_name"])# [["community_area_num", "community_area_name"]]

In [27]:
socio_life_merged_DF.head()

,community_area_num,community_area_name,housing_crowded_perc,housholds_below_poverty_perc,aged_16_or_more_unemployed_perc,aged_25_or_more_without_high_school_diploma_perc,aged_under_18_or_over_64_perc,per_capita_income,hardship_idx,life_exp_1990,lower_95_perc_CI_1990,upper_95_perc_CI_1990,life_exp_2000,lower_95_perc_CI_2000,upper_95_perc_CI_2000,life_exp_2010,lower_95_perc_CI_2010,upper_95_perc_CI_2010
0,1,rogers park,7.7,23.6,8.7,18.2,27.5,23939,39.0,70.9,69.9,71.9,73.1,72.2,74.1,77.3,76.3,78.2
1,2,west ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0,76.9,76.1,77.8,78.1,77.3,78.8,80.3,79.5,81.1
2,3,uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0,64.0,63.1,64.9,71.7,70.8,72.7,76.0,75.1,76.9
3,4,lincoln square,3.4,10.9,8.2,13.4,25.5,37524,17.0,74.2,73.1,75.4,76.8,75.8,77.8,80.5,79.3,81.6
4,5,north center,0.3,7.5,5.2,4.5,26.2,57123,6.0,73.4,72.1,74.7,77.9,76.6,79.1,81.5,80.1,82.8


In [28]:
# Load the areas dataframe 
areas_DF = gpd.read_file(cst.AREAS_PATH)

DriverError: ../data/Boundaries - Community Areas (current)/geo_export_3ffbc00d-9720-4c2b-b3c0-ed098b3b9ae7.shp: No such file or directory

In [29]:
# Clean the dataframe
areas_DF = clean_database.clean_areas_df(areas_DF)

areas_DF.head()

NameError: name 'areas_DF' is not defined

In [18]:
# First we complete the missing lng/lat pairs from the address

# TODO

# Insert the area number of all lat/lng pairs into a new column
#food_inspections_DF[cst.AREA_NUM] = food_inspections_DF.apply(lambda row: areas_handler.get_area_num_from_lng_lat(row['lat'], row['lng'], areas_DF), axis=1)

# must probably do astype(int) after

In [113]:
food_unknown_loc = food_inspections_DF[food_inspections_DF['lat'].isna()]

In [114]:
# No need for duplicate queries, so we just keep the unique addresses
unknown_locations = pd.DataFrame(food_unknown_loc['address'].unique(), columns=['address'])

# Retrieve 'lat' and 'lng' values of an address and append them to 'unknown_locations' dataframe
unknown_locations[['lat', 'lng']] = unknown_locations['address'][:10].apply(lambda addr: pd.Series(areas_handler.get_lat_lng_from_address(addr)))
unknown_locations[:10]



,address,lat,lng
0,3455-3459 S OGDEN AVE,41.863365,-87.689995
1,4000 N O'HARE FIELD,NaN,NaN
2,2009 S LAFIN ST,NaN,NaN
3,5516 S Maplewood (2532W) AVE,NaN,NaN
4,1010 S DELANO CT,41.867383,-87.632547
5,6237 S HALSTED PKWY,NaN,NaN
6,2911 W W 47TH ST,41.809065,-87.634585
7,150 RIVERSIDE PLZ,41.882390,-87.638770
8,2011 N GRIFFIN BLVD,NaN,NaN
9,65 CARMINE ST,NaN,NaN


In [ ]:
# 4000 N O'HARE FIELD not recognized, but 4000 O'HARE is.
# 2009 S LAFIN ST : should be lafLin street, not lafin street...
# 1010 S DELANO CT is recognized by OSM, but removing the 'S' makes it unknown
# PKWY : PARKWAY
# BLVD : BOULEVARD

<a id = 'basic-stats'></a>
## Basic Statistics

We report some statistics on the various dataframes.

[Table of Contents](#up)

In [82]:
corr = socio_life_merged_DF[cst.SOCIOECONOMIC_METRICS].corr()
corr

,housing_crowded_perc,housholds_below_poverty_perc,aged_16_or_more_unemployed_perc,aged_25_or_more_without_high_school_diploma_perc,per_capita_income,hardship_idx,life_exp_2010,aged_under_18_or_over_64_perc
housing_crowded_perc,1.000000,0.319403,0.165299,0.875959,-0.541730,0.649574,-0.044064,0.224692
housholds_below_poverty_perc,0.319403,1.000000,0.800084,0.424294,-0.567025,0.803267,-0.691029,0.435894
aged_16_or_more_unemployed_perc,0.165299,0.800084,1.000000,0.355518,-0.656619,0.792294,-0.797766,0.676532
aged_25_or_more_without_high_school_diploma_perc,0.875959,0.424294,0.355518,1.000000,-0.709770,0.802538,-0.136151,0.408878
per_capita_income,-0.541730,-0.567025,-0.656619,-0.709770,1.000000,-0.849167,0.566589,-0.754844
hardship_idx,0.649574,0.803267,0.792294,0.802538,-0.849167,1.000000,-0.616442,0.690844
life_exp_2010,-0.044064,-0.691029,-0.797766,-0.136151,0.566589,-0.616442,1.000000,-0.566358
aged_under_18_or_over_64_perc,0.224692,0.435894,0.676532,0.408878,-0.754844,0.690844,-0.566358,1.000000


In [94]:
bad_metrics = set(['housing_crowded_perc', 'housholds_below_poverty_perc', 'aged_16_or_more_unemployed_perc', 
               'aged_25_or_more_without_high_school_diploma_perc', 'hardship_idx', 'aged_under_18_or_over_64_perc'])
good_metrics = set(['per_capita_income', 'life_exp_2010' ])
sign_kept = True

for c1 in cst.SOCIOECONOMIC_METRICS:
    for c2 in cst.SOCIOECONOMIC_METRICS:
        if (c1 in bad_metrics and c2 in bad_metrics) or (c1 in good_metrics and c2 in good_metrics):
            if corr[c][c] < 0:
                sign_kept = False
        elif (c1 in bad_metrics and c2 in good_metrics) or (c1 in good_metrics and c2 in bad_metrics):
            if corr[c][c] > 0:
                sign_kept = False
print(sign_kept)

True


In [86]:
#set correlation between each variable and itself to None in order to ignore it later
for c in corr.columns:
    corr[c][c] = None 
    
corrmax =pd.DataFrame(corr.idxmax()).rename({0: 'Strongest positive correlation'}, axis = 1)
corrmax['Correlation value'] = corr.max()
corrmax

,Strongest positive correlation,Correlation value
housing_crowded_perc,aged_25_or_more_without_high_school_diploma_perc,0.875959
housholds_below_poverty_perc,hardship_idx,0.803267
aged_16_or_more_unemployed_perc,housholds_below_poverty_perc,0.800084
aged_25_or_more_without_high_school_diploma_perc,housing_crowded_perc,0.875959
per_capita_income,life_exp_2010,0.566589
hardship_idx,housholds_below_poverty_perc,0.803267
life_exp_2010,per_capita_income,0.566589
aged_under_18_or_over_64_perc,hardship_idx,0.690844


In [87]:
corrmin =pd.DataFrame(corr.idxmin()).rename({0: 'Strongest negative correlation'}, axis = 1)
corrmin['Correlation value'] = corr.min()
corrmin

,Strongest negative correlation,Correlation value
housing_crowded_perc,per_capita_income,-0.541730
housholds_below_poverty_perc,life_exp_2010,-0.691029
aged_16_or_more_unemployed_perc,life_exp_2010,-0.797766
aged_25_or_more_without_high_school_diploma_perc,per_capita_income,-0.709770
per_capita_income,hardship_idx,-0.849167
hardship_idx,per_capita_income,-0.849167
life_exp_2010,aged_16_or_more_unemployed_perc,-0.797766
aged_under_18_or_over_64_perc,per_capita_income,-0.754844


Of the above correlations, we notice certain things: Firstly, we can classify the indicators between good (life expectancy and per capita income) and bad (percentage of crowded houses, percentage of below porverty households, percentage of over 16 unemployed people, percentage fo over 25 people without a high school diploma, the hardship index, and the percentage of people under 18 and over 64), and the correlation between indicators either both good or both bad will always be positive, whereas the correlation between a good and a bad indicator will always be negative. 

We also notice that the percentage of people under 18 or over 64 is a strong negative indicator: it is more negatively correlated to per capita income than, for example, the percentage of houses living below the poverty line. 

It is indeed quite surprising that per capita average income is not more correlated to the percentage of houses living below the poverty line (correlation is -0.56). We plot the 2 metrics in order to see this:

One reason the linear correlation is so low is that the relationship is exponential. Also, the top 5 highest per capita neighbourhoods are not in the top 15 lowest poor households percentage. TODO: why does this happen??'?!!! where (very downtown). What are some other indicators in this 'mixed' (rich and poor people) neighbourhoods?? This is a cool direction to go in i think

In [127]:
scatter = vincent.Scatter(socio_life_merged_DF[['per_capita_income','housholds_below_poverty_perc']], iter_idx = 'housholds_below_poverty_perc')

In [129]:
scatter.axis_titles(x='Percentage of households below the poverty line', y='per_capita_income')


TODO: You can not click in the above plot. It'd be cool to be able to click and see the name of a neighbourhood. How can we do this with vincent? I kknow how with plotly but they said to use vincent. 

CONTINUATION: finish socioeconomic things (average, stard deviation, max min. How many people live in poor areas, how many people live in bad areas, etc.). Maybe classify areas in 4 manually. Plot where al this is.

Once socioeconomic things are done, let's look at food things (inspections: where are they happening)?

Finally, look at correlation between the 2.
